# 

In [1]:
# Why prefer LLamaIndex? It's got nicer high-level objects to map from functions -> agents


In [16]:
# Let's start small

from pydantic.v1 import BaseModel, create_model
from typing import List
import inspect

from llama_index.program import OpenAIPydanticProgram
from llama_index.tools.function_tool import FunctionTool

In [17]:
from openbb import obb
from openbb_core.app.model.custom_parameter import OpenBBCustomParameter

In [18]:
from typing import Annotated, get_type_hints, _AnnotatedAlias
from pydantic.v1.fields import FieldInfo

def add(x: Annotated[int, "the first number"], y: Annotated[int, "the second number"]) -> int:
    "Add two numbers togther."
    print("Hoooory shiiit")
    return x + y

In [19]:
def get_param_info(param):
    param_annotation_description = None
    param_default_value = None
    
    if isinstance(param.annotation, _AnnotatedAlias):
        param_type = param.annotation.__args__[0] 
        param_annotation_description = param.annotation.__metadata__[0]
        if isinstance(param_annotation_description, OpenBBCustomParameter):
            param_annotation_description = param_annotation_description.description
    else: 
        param_type = param.annotation
    if param.default != inspect.Parameter.empty:
        param_default_value = param.default
    else:
        param_default_value = ...
        
    return {
        "type": param_type,
        "description": param_annotation_description,
        "default": param_default_value
    }

In [20]:
def process_params(func):
    parameters = inspect.signature(func).parameters
    all_param_info = {}
    for name, param in parameters.items():
        if name != "kwargs":  # TODO: Figure out how to handle **kwargs
            all_param_info[name] = get_param_info(param)
    return all_param_info

def map_params_to_pydantic_fields(params_info: dict):
    fields = {}
    for param_name, param_info in params_info.items():
        fields[param_name] = (
            param_info['type'],
            FieldInfo(
                description=param_info['description'],
                default=param_info['default']
            )
        )
    return fields

def make_pydantic_v1_model_from_func(func):
    param_info = process_params(func)
    fields = map_params_to_pydantic_fields(param_info)
    return create_model(func.__name__ + "Model", **fields)

In [21]:
m = make_pydantic_v1_model_from_func(obb.equity.fundamental.cash)
m.schema()

{'title': 'cashModel',
 'type': 'object',
 'properties': {'symbol': {'title': 'Symbol',
   'description': 'Symbol to get data for.',
   'anyOf': [{'type': 'string'},
    {'type': 'array', 'items': {'type': 'string'}}]},
  'period': {'title': 'Period',
   'description': 'Time period of the data to return.',
   'default': 'annual',
   'enum': ['annual', 'quarter'],
   'type': 'string'},
  'limit': {'title': 'Limit',
   'description': 'The number of data entries to return.',
   'default': 5,
   'type': 'integer'},
  'provider': {'title': 'Provider',
   'enum': ['fmp', 'intrinio', 'polygon'],
   'type': 'string'}},
 'required': ['symbol']}

In [22]:
from llama_index.tools.function_tool import FunctionTool
from llama_index.agent import OpenAIAgent
from llama_index.llms import OpenAI

In [25]:
from openbb_core.app.router import CommandMap

In [26]:
cm = CommandMap()

In [82]:
members = inspect.getmembers(obb.equity.fundamental)
members_select = [m for m in members if inspect.ismethod(m[1]) and '__' not in m[0] and '_run' not in m[0]]
callables = [m[1] for m in members_select]

In [90]:
f = callables[0]

AttributeError: 'function' object has no attribute '__file__'

In [37]:
# Let's do tool retrieval

fundamentals = callables

In [102]:
# Create a map to find providerinterfaces
cm = CommandMap()
pi = ProviderInterface()
lookup = {}
for f in callables:
    # TODO: Don't template in the router path
    lookup[f.__name__] = cm.commands_model[f"/equity/fundamental/{f.__name__}"]

# Then we get the provider interface +  keys from the lookup
provider_lookup = {}
for name, provider in lookup.items():
    provider_lookup[name] = pi.map[provider]

In [80]:
from openbb_core.app.provider_interface import ProviderInterface

cm = CommandMap()
pi = ProviderInterface()
cm.commands_model
pi.map['FinancialRatios']['openbb']['Data']['fields'].keys()


dict_keys(['symbol', 'date', 'period', 'current_ratio', 'quick_ratio', 'cash_ratio', 'days_of_sales_outstanding', 'days_of_inventory_outstanding', 'operating_cycle', 'days_of_payables_outstanding', 'cash_conversion_cycle', 'gross_profit_margin', 'operating_profit_margin', 'pretax_profit_margin', 'net_profit_margin', 'effective_tax_rate', 'return_on_assets', 'return_on_equity', 'return_on_capital_employed', 'net_income_per_ebt', 'ebt_per_ebit', 'ebit_per_revenue', 'debt_ratio', 'debt_equity_ratio', 'long_term_debt_to_capitalization', 'total_debt_to_capitalization', 'interest_coverage', 'cash_flow_to_debt_ratio', 'company_equity_multiplier', 'receivables_turnover', 'payables_turnover', 'inventory_turnover', 'fixed_asset_turnover', 'asset_turnover', 'operating_cash_flow_per_share', 'free_cash_flow_per_share', 'cash_per_share', 'payout_ratio', 'operating_cash_flow_sales_ratio', 'free_cash_flow_operating_cash_flow_ratio', 'cash_flow_coverage_ratios', 'short_term_coverage_ratios', 'capital_e

In [111]:
x = '\n - '.join(provider_lookup['balance']['openbb']['Data']['fields'].keys())
print(x)

symbol
 - date
 - cik
 - currency
 - filling_date
 - accepted_date
 - period
 - cash_and_cash_equivalents
 - short_term_investments
 - long_term_investments
 - inventory
 - net_receivables
 - marketable_securities
 - property_plant_equipment_net
 - goodwill
 - assets
 - current_assets
 - other_current_assets
 - intangible_assets
 - tax_assets
 - non_current_assets
 - other_non_current_assets
 - account_payables
 - tax_payables
 - deferred_revenue
 - other_assets
 - total_assets
 - long_term_debt
 - short_term_debt
 - liabilities
 - other_current_liabilities
 - current_liabilities
 - total_liabilities_and_total_equity
 - other_non_current_liabilities
 - non_current_liabilities
 - total_liabilities_and_stockholders_equity
 - other_stockholder_equity
 - total_stockholders_equity
 - other_liabilities
 - total_liabilities
 - common_stock
 - preferred_stock
 - accumulated_other_comprehensive_income_loss
 - retained_earnings
 - minority_interest
 - total_equity


In [184]:
# Now we need to define the tools

# Let's create a wrapper (aka decorator, but we won't use the sugar) to always return the `results` from an `OBBject`
def return_results(func):
    def wrapped_func(*args, **kwargs):
        return func(*args, **kwargs).to_df().to_json(orient="records")
    return wrapped_func

# Let's get the actual 
def make_function_tool(func):
    wrapped_func = return_results(func)
    fn_schema = make_pydantic_v1_model_from_func(func)
    first_line_docstring = func.__doc__.split('\n')[0]

    output_fields = provider_lookup[func.__name__]['openbb']['Data']['fields'].keys()
    output_fields_str = ','.join(output_fields) 

    description = f"{first_line_docstring}\n\n"
    description += "The following fields are returned in the output:\n"
    description += f"- {output_fields_str}"
    
    function_tool = FunctionTool.from_defaults(
        name=func.__name__,
        fn=wrapped_func,
        fn_schema=fn_schema,
        description=description,
    )
    return function_tool

In [185]:
all_tools = [make_function_tool(func) for func in fundamentals] 
all_tools_map = {t.metadata.name: t for t in all_tools}

all_tools[0].metadata.description

'Balance Sheet. Balance sheet statement.\n\nThe following fields are returned in the output:\n- symbol,date,cik,currency,filling_date,accepted_date,period,cash_and_cash_equivalents,short_term_investments,long_term_investments,inventory,net_receivables,marketable_securities,property_plant_equipment_net,goodwill,assets,current_assets,other_current_assets,intangible_assets,tax_assets,non_current_assets,other_non_current_assets,account_payables,tax_payables,deferred_revenue,other_assets,total_assets,long_term_debt,short_term_debt,liabilities,other_current_liabilities,current_liabilities,total_liabilities_and_total_equity,other_non_current_liabilities,non_current_liabilities,total_liabilities_and_stockholders_equity,other_stockholder_equity,total_stockholders_equity,other_liabilities,total_liabilities,common_stock,preferred_stock,accumulated_other_comprehensive_income_loss,retained_earnings,minority_interest,total_equity'

In [198]:
# Let's build an object index
from llama_index import VectorStoreIndex
from llama_index.objects import ObjectIndex, SimpleToolNodeMapping
from llama_index.tools import QueryEngineTool
from llama_index.query_engine import SubQuestionQueryEngine
from llama_index.query_engine.router_query_engine import RouterQueryEngine

tool_mapping = SimpleToolNodeMapping.from_objects(all_tools)
obj_index = ObjectIndex.from_objects(
    all_tools,
    tool_mapping,
    VectorStoreIndex
)

In [189]:
from llama_index.agent import FnRetrieverOpenAIAgent, OpenAIAgent, ReActAgent
from llama_index.llms import OpenAI
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=OpenAI("gpt-4-1106-preview"),
)

In [208]:
object_retriever = obj_index.as_retriever(similarity_top_k=3)

In [210]:
agent = ReActAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    verbose=True
)

In [211]:
response = agent.chat("What is the current revenue of TSLA? Use multiple tools if you don't find the necessary data.")
print(response)

Thought: I can use the multiples tool to get the revenue data for TSLA.
Action: multiples
Action Input: {'symbol': 'TSLA'}
Observation: [{"revenue_per_share_ttm":30.2027707809,"net_income_per_share_ttm":3.3866498741,"operating_cash_flow_per_share_ttm":3.8299748111,"free_cash_flow_per_share_ttm":1.1690806045,"cash_per_share_ttm":8.2106423174,"book_value_per_share_ttm":16.8343828715,"tangible_book_value_per_share_ttm":16.8828715365,"shareholders_equity_per_share_ttm":16.8343828715,"interest_debt_per_share_ttm":1.423488665,"market_cap_ttm":742563907850.0,"enterprise_value_ttm":731024907850.0,"pe_ratio_ttm":68.9737671997,"price_to_sales_ratio_ttm":7.7411691323,"pocf_ratio_ttm":60.9899572509,"pfcf_ratio_ttm":199.9902795179,"pb_ratio_ttm":13.8757685258,"ptb_ratio_ttm":13.8757685258,"ev_to_sales_ttm":7.6208759836,"enterprise_value_over_ebitda_ttm":44.7082690875,"ev_to_operating_cash_flow_ttm":60.0974110367,"ev_to_free_cash_flow_ttm":196.8825499192,"earnings_yield_ttm":0.0144982657,"free_cash_

In [ ]:
tool_mapping = SimpleTool

In [477]:
fn = obb.equity.fundamental.balance

def meta_fn(func):
    def wrapped_func(*args, **kwargs):
        return func(*args, **kwargs).results
    return wrapped_func

do_this = meta_fn(fn)

function_tool = FunctionTool.from_defaults(
    fn=do_this,
    fn_schema=make_pydantic_v1_model_from_func(fn),
    description="Balance sheet statement."
)

In [480]:
x = fn("TSLA")

In [481]:
tools = [function_tool]

In [482]:
llm = OpenAI("gpt-4-1106-preview")

In [483]:
agent = OpenAIAgent.from_tools(tools, verbose=True, llm=OpenAI("gpt-4-1106-preview"))

In [486]:
response = agent.chat("How much debt did TSLA have in 2022? Use function calling.")
print(response)

STARTING TURN 1
---------------

=== Calling Function ===
Calling function: wrapped_func with args: {"symbol":"TSLA","period":"annual","limit":1}
Got output: [FMPBalanceSheetData(symbol=TSLA, date=2022-12-31, cik=0001318605, currency=USD, filling_date=None, accepted_date=2023-01-30 21:29:15, period=FY, cash_and_cash_equivalents=16253000000.0, short_term_investments=5932000000.0, long_term_investments=0.0, inventory=12839000000.0, net_receivables=2952000000.0, marketable_securities=None, property_plant_equipment_net=36635000000.0, goodwill=194000000.0, assets=82338000000.0, current_assets=40917000000.0, other_current_assets=2941000000.0, intangible_assets=593000000.0, tax_assets=0.0, non_current_assets=41421000000.0, other_non_current_assets=4193000000.0, account_payables=15255000000.0, tax_payables=1235000000.0, deferred_revenue=2810000000.0, other_assets=0.0, total_assets=None, long_term_debt=1597000000.0, short_term_debt=1502000000.0, liabilities=36440000000.0, other_current_liabilit

In [487]:
obb.equity.price.historical?

Signature:
obb.equity.price.historical(
    symbol: Annotated[Union[str, List[str]], OpenBBCustomParameter(description='Symbol to get data for.')],
    interval: Annotated[Optional[str], OpenBBCustomParameter(description='Time interval of the data to return.')] = '1d',
    start_date: Annotated[Union[datetime.date, NoneType, str], OpenBBCustomParameter(description='Start date of the data, in YYYY-MM-DD format.')] = None,
    end_date: Annotated[Union[datetime.date, NoneType, str], OpenBBCustomParameter(description='End date of the data, in YYYY-MM-DD format.')] = None,
    provider: Optional[Literal['fmp', 'intrinio', 'polygon']] = None,
    **kwargs,
) -> openbb_core.app.model.obbject.OBBject[List[Data]]
Docstring:
Equity Historical price. Load stock data for a specific ticker.

Parameters
----------
symbol : str
    Symbol to get data for.
interval : Optional[str]
    Time interval of the data to return.
start_date : Optional[datetime.date]
    Start date of the data, in YYYY-MM-DD f